In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers
!pip install datasets

     |████████████████████████████████| 4.0 MB 4.9 MB/s 
     |████████████████████████████████| 596 kB 51.5 MB/s 
     |████████████████████████████████| 77 kB 6.0 MB/s 
     |████████████████████████████████| 895 kB 51.6 MB/s 
     |████████████████████████████████| 6.6 MB 55.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 325 kB 5.1 MB/s 
     |████████████████████████████████| 212 kB 43.4 MB/s 
     |████████████████████████████████| 1.1 MB 49.5 MB/s 
     |████████████████████████████████| 136 kB 66.2 MB/s 
     |████████████████████████████████| 127 kB 54.0 MB/s 
     |████████████████████████████████| 94 kB 131 kB/s 
     |████████████████████████████████| 144 kB 50.3 MB/s 
     |████████████████████████████████| 271 kB 52.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib

In [ ]:
import pandas as pd
import numpy as np
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import random
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import torch.nn.functional as F
import csv
import ast

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/NLP_Proj/data_with_keywords_train.csv')
data

,Unnamed: 0,Unnamed: 0.1,title,body,num_comments,verdict,word_count,has_tldr,has_background,has_update,has_edit,keywords,title_key_word
0,1881,1881,aita for asking my boyfriend to drive me to th...,"i am very, very sick right now with the flu. i...",43.0,not the asshole,573,False,False,False,True,"['hospital', 'drive', 'boyfriend', 'asking', '...","['aita asking', 'boyfriend drive', 'hospital',..."
1,33358,33358,aita for buying a phone off of my close friend...,my friend let’s call her zoe & her ex let’s ca...,24.0,not the asshole,267,False,False,False,False,"['close friend ’', 'ex boyfriend', 'phone', 'b...","['buying phone', 'aita', 'friend', 'ex boyfrie..."
2,36512,36512,aita for not answering my phone to my mom duri...,3 months ago i was going to a sleepover with m...,20.0,asshole,232,False,False,False,False,"['sleepover', 'phone', 'mom', 'answering', 'ai...","['mom sleepover', 'aita answering', 'phone', '..."
3,28733,28733,aita for asking my girlfriend to straighten he...,so some backstory. my girlfriend has really be...,58.0,asshole,418,False,False,False,False,"['straighten', 'hair', 'girlfriend', 'asking',...","['girlfriend straighten', 'aita asking', 'hair..."
4,23625,23625,aita for telling my girlfriend to stop eating ...,"weird title, and this’ll sound fake, but it’s ...",1164.0,asshole,280,False,False,False,False,"['stop eating “ baby snacks ”?', 'telling', 'g...","['baby snacks', 'aita telling', 'girlfriend st..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
37507,19883,19883,aita for letting someone sit in my seat at the...,i went to watch a play with my girlfriend last...,86.0,asshole,507,False,False,False,False,"['letting someone sit', 'theatre', 'seat', 'ai...","['seat theatre', 'letting', 'aita', 'sit', 'se..."
37508,14730,14730,aita for venting in a kind of nasty way about ...,so i was hanging out with my friend and i star...,7.0,asshole,229,False,False,False,False,"['nasty way', 'venting', 'kind', 'classmates',...","['aita venting', 'way classmates', 'kind nasty..."
37509,4116,4116,wibta if i asked my neighbor to help with the ...,tl;dr at the bottom.\n\n\n**i will also likely...,56.0,asshole,463,False,False,False,True,"['wibta', 'tree', 'removing', 'neighbor', 'hom...","['neighbor help', 'wibta asked', 'cost removin..."
37510,14580,14580,aita for not wanting my fiancée to invite her ...,"my fiancée and i have been dating 3 years, eng...",115.0,asshole,222,False,False,False,False,"['wedding', 'wanting', 'stepdaughter', 'invite...","['stepdaughter wedding', 'invite', 'aita', 'wa..."


In [ ]:
data_truncated = data[data['verdict'].isin(['asshole', 'not the asshole'])]
len(data_truncated)

37512

In [ ]:
data_truncated.head()

,Unnamed: 0,Unnamed: 0.1,title,body,num_comments,verdict,word_count,has_tldr,has_background,has_update,has_edit,keywords,title_key_word
0,1881,1881,aita for asking my boyfriend to drive me to th...,"i am very, very sick right now with the flu. i...",43.0,not the asshole,573,False,False,False,True,"['hospital', 'drive', 'boyfriend', 'asking', '...","['aita asking', 'boyfriend drive', 'hospital',..."
1,33358,33358,aita for buying a phone off of my close friend...,my friend let’s call her zoe & her ex let’s ca...,24.0,not the asshole,267,False,False,False,False,"['close friend ’', 'ex boyfriend', 'phone', 'b...","['buying phone', 'aita', 'friend', 'ex boyfrie..."
2,36512,36512,aita for not answering my phone to my mom duri...,3 months ago i was going to a sleepover with m...,20.0,asshole,232,False,False,False,False,"['sleepover', 'phone', 'mom', 'answering', 'ai...","['mom sleepover', 'aita answering', 'phone', '..."
3,28733,28733,aita for asking my girlfriend to straighten he...,so some backstory. my girlfriend has really be...,58.0,asshole,418,False,False,False,False,"['straighten', 'hair', 'girlfriend', 'asking',...","['girlfriend straighten', 'aita asking', 'hair..."
4,23625,23625,aita for telling my girlfriend to stop eating ...,"weird title, and this’ll sound fake, but it’s ...",1164.0,asshole,280,False,False,False,False,"['stop eating “ baby snacks ”?', 'telling', 'g...","['baby snacks', 'aita telling', 'girlfriend st..."


In [ ]:
data_truncated = data_truncated[['title', 'body', 'verdict']]

In [ ]:
data_truncated = data_truncated.fillna('')
data_truncated

,title,body,verdict
0,aita for asking my boyfriend to drive me to th...,"i am very, very sick right now with the flu. i...",not the asshole
1,aita for buying a phone off of my close friend...,my friend let’s call her zoe & her ex let’s ca...,not the asshole
2,aita for not answering my phone to my mom duri...,3 months ago i was going to a sleepover with m...,asshole
3,aita for asking my girlfriend to straighten he...,so some backstory. my girlfriend has really be...,asshole
4,aita for telling my girlfriend to stop eating ...,"weird title, and this’ll sound fake, but it’s ...",asshole
...,...,...,...
37507,aita for letting someone sit in my seat at the...,i went to watch a play with my girlfriend last...,asshole
37508,aita for venting in a kind of nasty way about ...,so i was hanging out with my friend and i star...,asshole
37509,wibta if i asked my neighbor to help with the ...,tl;dr at the bottom.\n\n\n**i will also likely...,asshole
37510,aita for not wanting my fiancée to invite her ...,"my fiancée and i have been dating 3 years, eng...",asshole


In [ ]:
TOKENS_THRESHOLD = 200
def filter_fn(row):
    if len(row['body'].split()) > TOKENS_THRESHOLD:
        return True
    else:
        return False

data_truncated = data_truncated[data_truncated.apply(filter_fn, axis=1)]
data_truncated = data_truncated.reset_index()
data_truncated

,index,title,body,verdict
0,0,aita for asking my boyfriend to drive me to th...,"i am very, very sick right now with the flu. i...",not the asshole
1,1,aita for buying a phone off of my close friend...,my friend let’s call her zoe & her ex let’s ca...,not the asshole
2,2,aita for not answering my phone to my mom duri...,3 months ago i was going to a sleepover with m...,asshole
3,3,aita for asking my girlfriend to straighten he...,so some backstory. my girlfriend has really be...,asshole
4,4,aita for telling my girlfriend to stop eating ...,"weird title, and this’ll sound fake, but it’s ...",asshole
...,...,...,...,...
29328,37507,aita for letting someone sit in my seat at the...,i went to watch a play with my girlfriend last...,asshole
29329,37508,aita for venting in a kind of nasty way about ...,so i was hanging out with my friend and i star...,asshole
29330,37509,wibta if i asked my neighbor to help with the ...,tl;dr at the bottom.\n\n\n**i will also likely...,asshole
29331,37510,aita for not wanting my fiancée to invite her ...,"my fiancée and i have been dating 3 years, eng...",asshole


In [ ]:
data_small = data_truncated.sample(n=25000)
data_small

,index,title,body,verdict
14229,18211,aita for questioning a guy who happened to be ...,i am 1 of 4 senior managers who work for a ger...,asshole
25804,32998,aita for not acknowledging my father's existen...,so i ( f 17 ) refuse to talk to or look at my ...,not the asshole
6173,7911,aita for telling my bestfriend about my boyfri...,i (21m) lived my entire life in a very cons...,asshole
20987,26847,for keeping my ex' fish tank?,she got a dirty used fish tank (60 gallon hex)...,asshole
2290,2955,aita for making my 14 year old brother stay up...,since i got home from my sophomore year of col...,asshole
...,...,...,...,...
7437,9574,aita for not allowing a baby at my wedding?,my fiance and i are getting married in october...,not the asshole
12782,16382,wibta if i refuse to add my stepdaughter as a ...,daughter and sd are 14&15 respectively. wife a...,not the asshole
1954,2522,aita for disowning my cousin (who's also my wi...,\n\nmy cousin decided to marry her boyfriend...,not the asshole
10546,13523,aita for suggesting a budding artist to use mu...,"so, the sister, a of my friend, b (19a and 23b...",not the asshole


In [ ]:
data_small = data_small.reset_index(drop = True)
data_small = data_small[['title', 'body', 'verdict']]
data_small

,title,body,verdict
0,aita for questioning a guy who happened to be ...,i am 1 of 4 senior managers who work for a ger...,asshole
1,aita for not acknowledging my father's existen...,so i ( f 17 ) refuse to talk to or look at my ...,not the asshole
2,aita for telling my bestfriend about my boyfri...,i (21m) lived my entire life in a very cons...,asshole
3,for keeping my ex' fish tank?,she got a dirty used fish tank (60 gallon hex)...,asshole
4,aita for making my 14 year old brother stay up...,since i got home from my sophomore year of col...,asshole
...,...,...,...
24995,aita for not allowing a baby at my wedding?,my fiance and i are getting married in october...,not the asshole
24996,wibta if i refuse to add my stepdaughter as a ...,daughter and sd are 14&15 respectively. wife a...,not the asshole
24997,aita for disowning my cousin (who's also my wi...,\n\nmy cousin decided to marry her boyfriend...,not the asshole
24998,aita for suggesting a budding artist to use mu...,"so, the sister, a of my friend, b (19a and 23b...",not the asshole


In [ ]:
data_small['verdict'].value_counts()

not the asshole    12912
asshole            12088
Name: verdict, dtype: int64

In [ ]:
data_truncated['verdict'].value_counts()

not the asshole    15200
asshole            14133
Name: verdict, dtype: int64

In [ ]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained('gpt2', 
                                          bos_token='<|startofprompt|>', 
                                          eos_token='<|endoftext|>', 
                                          pad_token='<|pad|>',
                                         )


tokenizer.encode("Sample Text")

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


[36674, 8255]

In [ ]:
tokenizer.add_tokens(['<|endofprompt|>', '<|startofverdict|>', '<|endofverdict|>', '<|startoftext|>'], special_tokens=True)
tokenizer.convert_tokens_to_ids(['<|endofprompt|>', '<|startofverdict|>', '<|endofverdict|>', '<|startoftext|>'])

[50259, 50260, 50261, 50262]

In [ ]:
sample_title = data_small[:1]['title'].values[0]
sample_verdict = data_small[:1]['verdict'].values[0]
sample_body = data_small[:1]['body'].values[0]


print(sample_title)
print(sample_verdict)
print(sample_body)

aita for questioning a guy who happened to be black who was parked in the management zone at my work?
asshole
i am 1 of 4 senior managers who work for a german construction company in an office based in london, however i am american by nationality. 

i am lucky enough to be part of a hugely dynamic team and that involves meeting people from head office pretty often. we pull in huge figures annually as a team so usually we are pretty much a powerhouse of our own and in ho’s good graces all the time. 

this morning i followed a blacked out m5 into my workplace, which are not the cars that head office drives. they usually roll around in audi’s, and we are also given company audi’s to drive. he was in front of me as we entered the parking lot and i saw he buzzed in to the gatehouse to be let through. obviously he didn’t have an id pass which made me suspicious of him. he had been blaring music with a heavy bass, which is normal for london but absolutely not normal in my corporate environme

In [ ]:
bs = 1
max_story = 800

In [ ]:
titles = data_small.title.copy()
verdicts = data_small.verdict.copy()
stories = data_small.body.copy()

In [ ]:
import torch
torch.manual_seed(42)
from torch.utils.data import Dataset 

class AitaDataset(Dataset):

  def __init__(self, titles, verdicts, txt_list, tokenizer, gpt2_type="gpt2", max_length=max_story):

    self.tokenizer = tokenizer # the gpt2 tokenizer we instantiated
    self.input_ids = []
    self.attn_masks = []
    self.labels = []

    for i in range(len(txt_list)):
      """
      This loop will iterate through each entry in the text corpus.
      For each bit of text it will prepend it with the start of text token,
      then append the end of text token and pad to the maximum length with the 
      pad token. 
      """

      encodings_dict = tokenizer('<|startofprompt|>'+ titles[i] + "<|endofprompt|>" 
                                 + verdicts[i] + "<|endofverdict|>" + txt_list[i] + "<|endoftext|>",
                                 truncation=True, 
                                 max_length=max_length, 
                                 padding="max_length",
                                 )
      prompts = tokenizer("<|startofprompt|>" + titles[i] + "<|endofprompt|>" + verdicts[i] + "<|endofverdict|>", truncation=True, max_length=max_length)
      
      """
      Each iteration then appends either the encoded tensor to a list,
      or the attention mask for that encoding to a list. The attention mask is
      a binary list of 1's or 0's which determine whether the langauge model
      should take that token into consideration or not. 
      """
      self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
      self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))
      self.labels.append(torch.tensor(prompts["input_ids"]))
    
  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, idx):
    return self.input_ids[idx], self.labels[idx], self.attn_masks[idx] 

In [ ]:
from torch.utils.data import random_split

dataset = AitaDataset(titles, verdicts, stories, tokenizer, max_length=800)

# Split into training and validation sets
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

f'There are {train_size} samples for training, and {val_size} samples for validation testing'

'There are 22500 samples for training, and 2500 samples for validation testing'

In [ ]:
import random
from transformers import GPT2LMHeadModel, GPT2Config
import numpy as np

# Loading the model configuration and setting it to the GPT2 standard settings.
configuration = GPT2Config.from_pretrained('gpt2', output_hidden_states=False)

# Create the instance of the model and set the token size embedding length
model = GPT2LMHeadModel.from_pretrained("gpt2", config=configuration)
model.resize_token_embeddings(len(tokenizer))

# Tell pytorch to run this model on the GPU.
device = torch.device("cuda")
model.cuda()

# This step is optional but will enable reproducible runs.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

Downloading:   0%|          | 0.00/523M [00:00<?, ?B/s]

In [ ]:
epochs = 4
warmup_steps = 1e2
sample_every = 100

''' Initializing Optimizer '''
from transformers import AdamW
# AdamW is a class from the huggingface library, it is the optimizer we will be using, and we will only be instantiating it with the default parameters. 
optimizer = AdamW(model.parameters(),
                  lr = 5e-4,
                  eps = 1e-8
                )

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

train_dataloader = DataLoader(
            train_dataset,  
            sampler = RandomSampler(train_dataset), # Sampling for training is random
            batch_size = bs
        )

validation_dataloader = DataLoader(
            val_dataset, 
            sampler = SequentialSampler(val_dataset), # Sampling for validation is sequential as the order doesn't matter.
            batch_size = bs
        )

In [ ]:
from transformers import get_linear_schedule_with_warmup

"""
Total training steps is the number of data points, times the number of epochs. 
Essentially, epochs are training cycles, how many times each point will be seen by the model. 
"""

total_steps = len(train_dataloader) * epochs

"""
set a variable learning rate which will help scan larger areas of the 
problem space at higher LR earlier, then fine tune to find the exact model minima 
at lower LR later in training.
"""
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = warmup_steps, 
                                            num_training_steps = total_steps)

# **Perplexity**

In [ ]:
def calculate_perplexity(model, input_ids, attention_mask, pad_token, stride):
  max_len = model.config.n_positions
  loss_fn = torch.nn.CrossEntropyLoss()
  loss = []
  for i in range(0, input_ids.size(1), stride):
    begin = max(i+stride-max_len, 0)
    end = min(i+stride, input_ids.size(1))
    trg_len = end - i
    ip = input_ids[:, begin:end].to(device)
    mask = attention_mask[:, begin:end].to(device)
    target_ids = ip.clone()

    with torch.no_grad():
        output = model (ip,
                          labels=target_ids, 
                          attention_mask = mask,
                          token_type_ids=None
                        )
        l = output[0]*trg_len
    loss.append(l)
  
  ppl = torch.exp(torch.stack(loss).sum() / end)
  return ppl.item()

# **Sample Model**

In [ ]:
import random
import time
import datetime

def format_time(elapsed):
    return str(datetime.timedelta(seconds=int(round((elapsed)))))

total_t0 = time.time()

training_stats = []

model = model.to(device)

for epoch_i in range(0, epochs):

    print(f'Beginning epoch {epoch_i + 1} of {epochs}')

    t0 = time.time()

    total_train_loss = 0

    model.train()

    for step, batch in enumerate(train_dataloader):

        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_title = batch[1].to(device)
        b_masks = batch[2].to(device)

        model.zero_grad()        

        outputs = model(  b_input_ids,
                          labels=b_labels, 
                          attention_mask = b_masks,
                          token_type_ids=None
                        )
        loss = outputs[0]  

        batch_loss = loss.item()
        total_train_loss += batch_loss

        # Get sample every 100 batches.
        if step % sample_every == 0 and not step == 0:

            elapsed = format_time(time.time() - t0)
            print(f'Batch {step} of {len(train_dataloader)}. Loss:{batch_loss}. Time:{elapsed}')

            model.eval()
 
            print("Perplexity is : ", calculate_perplexity(model, b_input_ids, b_masks, tokenizer.convert_tokens_to_ids('<|pad|>') ,100))

            break
            
            model.train()

        loss.backward()

        optimizer.step()

        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)       
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print(f'Average Training Loss: {avg_train_loss}. Epoch time: {training_time}')

    t0 = time.time()

    model.eval()

    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[2].to(device)
        
        with torch.no_grad():        

            outputs  = model(b_input_ids,  
                             attention_mask = b_masks,
                             labels=b_labels)
          
            loss = outputs[0]  
            
        batch_loss = loss.item()
        total_eval_loss += batch_loss        

    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    validation_time = format_time(time.time() - t0)    

    print(f'Validation loss: {avg_val_loss}. Validation Time: {validation_time}')

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

    break
print(f'Total training took {format_time(time.time()-total_t0)}')

Beginning epoch 1 of 4
Batch 100 of 22500. Loss:1.4167519807815552. Time:0:00:17
Perplexity is :  12.539158821105957
Average Training Loss: 0.036682381561067366. Epoch time: 0:00:18
Validation loss: 2.2051464969158174. Validation Time: 0:02:13
Total training took 0:02:30


# **Prompt Ranking Accuracy**

In [ ]:
# Size of the test dataset
size = len(titles)

# Number of samples taken into consideration for prompt ranking accuracy
num_samples = 10
hit = 0
total = 0

# Model
model = model.to(device)


# Get the samples
get_indices = list(np.random.randint(size, size=num_samples))

for i in get_indices:
  found = 0
  sample_ten = []
  sample_prompts = []
  wrong_sample_index = []
  conditioned_input_lens = []
  story_len = []

  # Original Prompt + Original Story
  text = tokenizer("<|startofprompt|>"+ titles[i] + "<|endofprompt|>" 
                                 + verdicts[i] + "<|endofverdict|>" + stories[i] + "<|endoftext|>",
                                 truncation=True, 
                                 max_length=max_story, 
                                 padding="max_length",
                                 )
  sample_ten.append(text)
  conditioned_input_lens.append(len(("<|startofprompt|> "+ titles[i] + " <|endofprompt|> " 
                                    + verdicts[i] + " <|endofverdict|>").split()))
  story_len.append(len(text["input_ids"]))


  
  # Get the indices of other stories. Repeat until original data index is not found
  while not found:
    wrong_sample_index = list(np.random.randint(size, size=10))
    if i not in wrong_sample_index:
      found=1
  

  for j in wrong_sample_index:

    # Wrong prompt + Original story 
    text = tokenizer("<|startofprompt|>"+ titles[j] + "<|endofprompt|>" 
                                 + verdicts[j] + "<|endofverdict|>" + stories[i] + "<|endoftext|>",
                                 truncation=True, 
                                 max_length=max_story, 
                                 padding="max_length",
                                 )
    sample_ten.append(text)

    conditioned_input_lens.append(len(("<|startofprompt|> "+ titles[j] + " <|endofprompt|> " 
                                    + verdicts[j] + " <|endofverdict|>").split()))
    story_len.append(len(text["input_ids"]))
  

  ## Compute Log probs

  probs = []
  for j in range(10):
    with torch.no_grad():

      ## Reconstruct the new prompts and story
      outputs = model(torch.tensor(sample_ten[j]["input_ids"]).unsqueeze(0).to(device),
                      labels = torch.tensor(sample_ten[j]["input_ids"]).unsqueeze(0).to(device),
                      attention_mask = torch.tensor(sample_ten[j]["attention_mask"]).unsqueeze(0).to(device),
                      token_type_ids=None
                      )

    # Apply log softmax
    lprobs = torch.log_softmax(outputs[1], dim=-1)

    lprobs = lprobs.gather(-1, torch.tensor(text["input_ids"]).unsqueeze(0).to(device).unsqueeze(-1)).squeeze(-1)
    probs.append(lprobs)

    
  probs = torch.stack((probs))
  logls = []
  for i in range(len(probs)):
      log_probs = probs[i, 0, conditioned_input_lens[i]:story_len[i] - 1]
      logls.append(-log_probs.float().mean().item())
            
  if logls[0] == min(logls):
      hit += 1
  total += 1

print(f"The Prompt ranking accuracy is :{str((float(hit)/float(total)))}")

The Prompt ranking accuracy is :0.2
